In [ ]:
# -*- coding: utf-8 -*-
"""
@author: Sheng with some minor changes by Kyna
"""
import torch
import torch.nn as nn
from torch import cat
import torch.nn.init as init
import math
import sys
sys.path.append('Utils')

class AgeEncoding(nn.Module):
    "Implement the AE function."
    def __init__(self, d_model, dropout, out_dim,max_len=240):
        super(AgeEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp((torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model)).float())
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        self.fc6 = nn.Sequential()
        self.fc6.add_module('fc6_s1',nn.Linear(d_model,512))
        self.fc6.add_module('lrn0_s1',nn.LayerNorm(512))
        self.fc6.add_module('fc6_s3',nn.Linear(512, out_dim))

    def forward(self, x, age_id):
        '''modification: convert age_id into an integer index that falls within the range of max_len (240)
        by rounding and clamping age_id.'''
        # Convert age to an index within the range [0, max_len-1]
        age_index = age_id.round().clamp(0, self.pe.size(0) - 1).long()
        y = self.pe[age_index,:]
        y = self.fc6(y)

        x += y
        return self.dropout(x)
       
    #def forward(self, x, age_id):
        #y = torch.autograd.Variable(self.pe[age_id,:], 
                         #requires_grad=False)
        #y = self.fc6(y)

        #x += y
        #return self.dropout(x)

                         

    
class AgeEncoding_simple(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, out_dim,max_len=240):
        super(AgeEncoding_simple, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        self.fc6 = nn.Linear(out_dim+1,out_dim)
        
        
    def forward(self, x, age_id):
        age_id = (age_id -0)/(240 - 0 + 1e-6)
        y = torch.cat([x.float(),age_id.unsqueeze(-1).float()],dim=1)
        x = self.fc6(y)

        return self.dropout(x)



class NetWork(nn.Module):

    def __init__(self, in_channel=1,feat_dim=1024,expansion = 4, type_name='conv3x3x3', norm_type = 'Instance'):
        super(NetWork, self).__init__()
        

        self.conv = nn.Sequential()

        self.conv.add_module('conv0_s1',nn.Conv3d(in_channel, 4*expansion, kernel_size=1))

        if norm_type == 'Instance':
           self.conv.add_module('lrn0_s1',nn.InstanceNorm3d(4*expansion))
        else:
           self.conv.add_module('lrn0_s1',nn.BatchNorm3d(4*expansion))
        self.conv.add_module('relu0_s1',nn.ReLU(inplace=True))
        self.conv.add_module('pool0_s1',nn.MaxPool3d(kernel_size=3, stride=2))

        self.conv.add_module('conv1_s1',nn.Conv3d(4*expansion, 32*expansion, kernel_size=3,padding=0, dilation=2))
        
        if norm_type == 'Instance':
            self.conv.add_module('lrn1_s1',nn.InstanceNorm3d(32*expansion))
        else:
            self.conv.add_module('lrn1_s1',nn.BatchNorm3d(32*expansion))
        self.conv.add_module('relu1_s1',nn.ReLU(inplace=True))
        self.conv.add_module('pool1_s1',nn.MaxPool3d(kernel_size=3, stride=2))


        
        
        self.conv.add_module('conv2_s1',nn.Conv3d(32*expansion, 64*expansion, kernel_size=5, padding=2, dilation=2))
        
        if norm_type == 'Instance':
            self.conv.add_module('lrn2_s1',nn.InstanceNorm3d(64*expansion))
        else:
            self.conv.add_module('lrn2_s1',nn.BatchNorm3d(64*expansion))
        self.conv.add_module('relu2_s1',nn.ReLU(inplace=True))
        self.conv.add_module('pool2_s1',nn.MaxPool3d(kernel_size=3, stride=2))

        
        self.conv.add_module('conv3_s1',nn.Conv3d(64*expansion, 64*expansion, kernel_size=3, padding=1, dilation=2))
        
        if norm_type == 'Instance':
            self.conv.add_module('lrn3_s1',nn.InstanceNorm3d(64*expansion))
        else:
            self.conv.add_module('lrn2_s1',nn.BatchNorm3d(64*expansion))
        self.conv.add_module('relu3_s1',nn.ReLU(inplace=True))
        self.conv.add_module('pool2_s1',nn.MaxPool3d(kernel_size=5, stride=2))

        self.fc6 = nn.Sequential()
        self.fc6.add_module('fc6_s1',nn.Linear(64*expansion*5*5*5, feat_dim))
        self.age_encoder = AgeEncoding(512,0.1,feat_dim)


    def load(self,checkpoint):
        model_dict = self.state_dict()
        pretrained_dict = torch.load(checkpoint)['state_dict']
        pretrained_dict = {k[6:]: v for k, v in list(pretrained_dict.items()) if k[6:] in model_dict and 'conv3_s1' not in k and 'fc6' not in k and 'fc7' not in k and 'fc8' not in k}

        model_dict.update(pretrained_dict)
        

        self.load_state_dict(model_dict)
        print([k for k, v in list(pretrained_dict.items())])
        return pretrained_dict.keys()

    def freeze(self, pretrained_dict_keys):
        for name, param in self.named_parameters():
            if name in pretrained_dict_keys:
                param.requires_grad = False
                

    def save(self,checkpoint):
        torch.save(self.state_dict(), checkpoint)
    
    #def forward(self, x, age_id):

        #z = self.conv(x)
        #z = self.fc6(z.view(x.shape[0],-1))
        #if age_id is not None:
            #z = self.age_encoder(z,age_id)
        #return z
        
    def forward(self, x, age_id):
        z = self.conv(x)
        z = self.fc6(z.view(x.shape[0], -1))

        # Check if age_id is not None and use AgeEncoding
        if age_id is not None:
            z = self.age_encoder(z, age_id)
        return z


def weights_init(model):
    if type(model) in [nn.Conv3d,nn.Linear]:
        nn.init.xavier_normal_(model.weight.data)
        nn.init.constant_(model.bias.data, 0.1)


In [ ]:
#WARNING: RUN AT YOUR OWN RISK, TAKES HALF A DAY ON MACBOOK

#Converts DICOM folders to NiFti images with SimpleITK
import SimpleITK as sitk
import os
import re

#Resample image if the spacing isn't consistent
def resample_image(image, new_spacing):
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()

    new_size = [
        int(round(osz*ospc/nspc)) for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)
    ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)

    return resampler.Execute(image)




def remove_artifacts(image):
'''remove artifacts for an image proccessed.
This is assuming the artifact has a significantly different intensity than the brain,
which is not be really the case'''
    
    # we can calculate the threshold using Otsu's method
    otsu_filter = sitk.OtsuThresholdImageFilter()
    otsu_filter.Execute(image)
    threshold = otsu_filter.GetThreshold()
    
    # Create an image filled with the threshold value + make sure to match the size and pixel type
    threshold_image = sitk.Image(image.GetSize(), image.GetPixelIDValue())
    threshold_image.CopyInformation(image)
    threshold_image += threshold  # Broadcasting the threshold value to all pixels
    
    #make a binary 'mask' where true values indicate the presence of the brain
    brain_mask = sitk.Greater(image, threshold_image)
    
    #Apply the mask to the image
    image = sitk.Mask(image, brain_mask, outsideValue=0)

    return image




def convert_dicom_to_nifti(dicom_folder, output_folder, target_spacing=1.2):

    
    #Make a list of all DICOM file paths in the folder
    dicom_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(dicom_folder)
    
    # Exclude the first 15 and the last 15 images (some files start and end with 15 slices of random noise, might be what's creating artifacts)
    #For simplicity and consistency, do the exclusion for all folders
    dicom_names = dicom_names[15:-15]

    #Check if there are DICOM files in the folder
    if not dicom_names:
        print(f"No DICOM files found in the folder: {dicom_folder}")
        return

    #Initialize the reader
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(dicom_names)

    #Execute the reader (to load the DICOM series)
    image = reader.Execute()

    # Resample if we need (i.e. if the spacing doesn't match the target spacing)
    current_spacing = image.GetSpacing()
    if current_spacing[2] != target_spacing: #should i explicitly set target spacing to 1.2? 
        new_spacing = (current_spacing[0], current_spacing[1], target_spacing)
        image = resample_image(image, new_spacing)

    #PermuteAxes to change the axes of the data
    image = sitk.PermuteAxes(image, [2, 1, 0])

    #remove artifacts
    #image = remove_artifacts(image)

    # Extracting the name for the NIFTI file from one of the DICOM files in the folder (naming is consistent throughout a folder)
    sample_name = os.path.basename(dicom_names[0])
    parts = sample_name.split('_')
    nifti_name = '_'.join(parts[1:4] + [parts[9]] + parts[-1].split('.'))
    nifti_name = re.sub(r'\.dcm$', '', nifti_name) + '.nii.gz'

    # Specify the output file path for the nifti
    output_file_path = os.path.join(output_folder, nifti_name)

    # Write the image to the NIFTI file
    sitk.WriteImage(image, output_file_path)

    print(f"Conversion complete. NIFTI file saved as {output_file_path}")



#walk through directories, if finds dicom, then take all files in that folder and convert to one nifti
def traverse_directories(root_folder, output_folder):
    for subdir, dirs, files in os.walk(root_folder):
        if any(f.endswith('.dcm') for f in files):
            convert_dicom_to_nifti(subdir, output_folder)

root_dir = '/Users/kynacrumley/Desktop/Fall2023.nosync/ADNI_DATA/ADNI'
output_dir = '//Users/kynacrumley/Desktop/Fall2023.nosync/ADNI_DATA/ADNI_NIITEST'
traverse_directories(root_dir, output_dir)


In [ ]:
#WARNING: RUN AT YOUR OWN RISK, TAKES MULTIPLE HOURS

#Evaluates all images in all subdirectories in the root folder through the CNN.
import torch
import nibabel as nib
import pandas as pd
import os
import re
import sys
import skimage.transform as skTrans

# Initialize pretrained model
device = torch.device('cpu')

checkpoint_path = '/Users/kynacrumley/Desktop/Fall2023.nosync/SDS3386/Term_Project/age_expansion_8_model_low_loss.pth.tar'
checkpoint = torch.load(checkpoint_path, map_location=device)
model_state_dict = checkpoint['state_dict']

# So that I can run the from models import NetWork as a .py in my terminal
module_path = '/Users/kynacrumley/Desktop/Fall2023.nosync/SDS3386/Term_Project'
sys.path.append(module_path)
from modelsv2 import NetWork

model = NetWork(in_channel=3, feat_dim=1024)  # should be 1024

# Load the reference sheet
ref_df = pd.read_csv('/Users/kynacrumley/Desktop/PTReferenceSheet.csv')

# Extract Features from all files in a folder.
results = []
root_path = '/Users/kynacrumley/Desktop/Fall2023.nosync/ADNI_DATA/ADNI1-3Raw/ADNI_Raw'

# Let's keep track of files that aren't processed correctly
unprocessed_files = []

# Regular expression to extract subject ID
id_pattern = re.compile(r'([A-Z]\d+)_ADNI_')

# Traverse directories and process .nii files
for subdir, dirs, files in os.walk(root_path):
    for file_name in files:
        if file_name.endswith('.nii'):
            print(f"Processing file: {file_name}")  # Print the file being processed

            match = id_pattern.match(file_name)
            if match:
                try:
                    unique_id = match.group(1)  # Unique ID (ex. I100021)

                    ref_info = ref_df[ref_df['UniqueID'] == unique_id]
                    if not ref_info.empty:
                        session_date = ref_info['SessionId'].iloc[0]
                        age = ref_info['EstimatedAge'].iloc[0]
                        age_tensor = torch.tensor([age], dtype=torch.float32).to(device)
                    else:
                        # If age is not found, set age_tensor to None
                        age_tensor = None
                        session_date = "Unknown"  # Handle unknown session date

                    image_path = os.path.join(subdir, file_name)

                    # Load image
                    image = nib.load(image_path)
                    data = image.get_fdata()
                    data = skTrans.resize(data, (3, 100, 100, 100), order=1, preserve_range=True)
                    data = (data - data.mean()) / data.std()
                    data = torch.tensor(data, dtype=torch.float32).unsqueeze(0).to(device)

                    # Forward pass through the model
                    output = model(data, age_id=age_tensor).squeeze().detach().cpu().numpy()

                    # Create a dictionary for each image
                    image_features = {'UniqueID': unique_id, 'SessionId': session_date}
                    for i, value in enumerate(output):
                        feature_key = f'Feature_{i + 1}'
                        image_features[feature_key] = value

                    results.append(image_features)

                    print('Processed successfully:', file_name)
                except Exception as e:
                    print(f"Error processing {file_name}: {e}")
                    unprocessed_files.append({'FileName': file_name, 'Error': str(e)})
            else:
                print(f"File name pattern does not match: {file_name}")
                unprocessed_files.append({'FileName': file_name, 'Error': 'Pattern Mismatch'})

# Create DataFrames from the results and unprocessed files
df = pd.DataFrame(results)
df_unprocessed = pd.DataFrame(unprocessed_files)

# Save to CSV
df.to_csv("ADNI123_051223KC_Features_Wide.csv", index=False)
df_unprocessed.to_csv("Unprocessed_Files.csv", index=False)

In [ ]:
import pandas as pd
data = pd.read_csv('ADNI123_051223KC_Features_Wide.csv')
meta = pd.read_csv('MPRAGEMETA_02Dec2023.csv')
diag = pd.read_csv('DXSUM_PDXCONV_ADNIALL_01Dec2023.csv')

#rename columns to match between dataframes
meta = meta.rename(columns = {'ImageUID':'UniqueID','ScanDate':'SessionId'})

#drop unnecessary columns in meta
meta = meta.drop(columns = ['Orig/Proc','Visit','MagStrength','Sequence','StudyID','SeriesID'])

#make sure they bot have SessionId in the same format:
data = data.drop(columns = ["SessionId"]) #seems that the dates don't match -> some dcm file names have dates that are not the same as their enclosing folder (the enclosing folder date matches other metadata such as in mpragemeta.csv)

#handle duplicates

duplicates = data[data['UniqueID'].duplicated(keep=False)]
print(duplicates)

data = data.drop_duplicates(subset='UniqueID', keep='first')

#make sure they both have UniqueID in the same format (need to add an I to the meta one)
meta['UniqueID'] = "I" + meta['UniqueID'].astype('string')

#inner merge
df = data.merge(meta, on=['UniqueID'], how='inner')
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]  # Move the last two columns to the front
df = df[cols]

#Add diagnosis by matching nearest date
import pandas as pd
from datetime import datetime

def find_closest_dates2(df1, df2):
    # Ensure the date columns in both dataframes are in datetime format
    df1['SessionId'] = pd.to_datetime(df1['SessionId'])
    df2['SessionId'] = pd.to_datetime(df2['SessionId'])

    # Function to find the closest date for a given subject
    def get_closest_date(subject_id, session_date):
        # Filter df2 to obtain only rows corresponding to the same subjectId
        subject_dates = df2[df2['SubjectID'] == subject_id]['SessionId']

        # Check if there are any dates for this subject
        if not subject_dates.empty:
            # Find the closest date in subject_dates to the session_date
            closest_date = min(subject_dates, key=lambda x: abs(x - session_date))
            return closest_date
        else:
            # if no dates available return none
            print("returning None for ", subject_id, session_date)
            return None

    # Apply the get_closest_date function to each row in df1
    df1['ClosestDate'] = df1.apply(lambda row: get_closest_date(row['SubjectID'], row['SessionId']), axis=1)

    return df1

#keep only necessary columns
diag = diag[['PTID','VISDATE','DXCURREN']]

#rename them to match
diag = diag.rename(columns = {'PTID':'SubjectID','VISDATE':'SessionId'})
result = find_closest_dates2(df, diag)


#now we can add the diagnosis column by matching the closest date
merged_df = pd.merge(result, diag, left_on='ClosestDate', right_on='SessionId', how='left')
merged_df = merged_df.drop(columns = ['SubjectID_y','SessionId_y'])

merged_df['scan_ID'] = merged_df['SubjectID_x'].str.replace('_','')

features = [f'Feature_{i}' for i in range(1, 1025)]
df_unique = merged_df.drop_duplicates(subset=features)

#Add source column

test_df = pd.read_csv('Test_diagnosis_ADNI.tsv', sep = '\t')
train_df = pd.read_csv('Train_diagnosis_ADNI.tsv', sep = '\t')
val_df = pd.read_csv('Val_diagnosis_ADNI.tsv', sep = '\t')

train_df['source'] = 'train'
test_df['source'] = 'test'
val_df['source'] = 'val'

source_df = pd.concat([train_df, test_df, val_df], ignore_index=True)
source_df = source_df[['participant_id', 'source']]
source_df = source_df.drop_duplicates()

df_unique['participant_id'] = 'sub-ADNI' + df_unique['SubjectID_x'].str.replace('_', '')


final = df_unique.merge(source_df, on='participant_id', how='left')
final = final.drop_duplicates()

final['source'] = final['source'].fillna('unseen')
final = final.dropna()
final = final[final['SubjectID_x'] != '133_S_0488'] #this is an outlier, conversion didn't work.


#Now reorder and rename columns so that my teammates can more easily use this dataset

cols = final.columns.tolist()
cols = cols[-5:] + cols[:-5]  # Move the last five columns to the front
final = final[cols]

final = final.rename(columns = {'DXCURREN':'Diagnosis','SubjectID_x':'SubjectID','SessionId_x':'SessionId','ClosestDate':'DiagnosisVisDate'})

cols = final.columns.tolist()
newcols = ['scan_ID','Diagnosis','source','SubjectID','SessionId','DiagnosisVisDate'] +cols[7:]
final = final[newcols]

# Count the number of "None" values in the 'Column1'
unknown_count = final['SessionId'].value_counts(dropna=False).get("Unknown", 0)

print("Number of 'None' values in 'Column1':", unknown_count)

final.to_csv('ADNI_Features_1-3v4.csv', index = False)


In [ ]:
#T-SNE of MS / AD / Parkinson's Data

#All imports
import torch
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import skimage.transform as skTrans
import altair as alt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import umap

#Initialize pretrained model
device = torch.device('cpu')
checkpoint = torch.load('age_expansion_8_model_low_loss.pth.tar', map_location=device)
model_state_dict = checkpoint['state_dict']
from models import NetWork
model = NetWork(in_channel=3, feat_dim=1024) #should be 1024


#Extract Features from all files in a folder.

results = []
root_path = 'T1_Files'

# Need a regular expression to extract patient ID and diagnosis from filenames
id_pattern = re.compile(r'(\d{4})_\d+?_([A-Z]{2})\.nii')

# Go through all files in the directory
for file_name in os.listdir(root_path):
    
    # Use the regular expression to extract the patient ID and diagnosis from the filename
    match = id_pattern.match(file_name)
    
    if match:
        patient_id = int(match.group(1))  # Convert ID to integer to remove leading zeros
        diagnosis = match.group(2)
        
        image_path = os.path.join(root_path, file_name)
        
        # Load image
        image = nib.load(image_path)
        data = image.get_fdata()
        
        data = skTrans.resize(data, (3,100,100,100), order=1, preserve_range=True)
        
        data = (data - data.mean()) / data.std()
        
        # Convert to PyTorch tensor and adjust dimensions
        data = torch.tensor(data, dtype=torch.float32).unsqueeze(0)
        data = data.to(device)
        
        # Forward pass through the model
        output = model(data, age_id=None).squeeze().detach().cpu().numpy()

        # Store results
        for index, value in enumerate(output):
            results.append({'PatientID': patient_id, 'Diagnosis': diagnosis, 'FeatureValue': value.item(), 'FeatureIndex': index})

# Create a Pandas DataFrame from the dic of results
df = pd.DataFrame(results)

print(df)
df.to_excel("MS_AD_HC_Features.xlsx")  #should specify index = False next time

#Let's decide how many PCA components we want


# Pivot the DataFrame to get one row per patient with all features
df_pivot = df.pivot(index='PatientID', columns='FeatureIndex', values='FeatureValue')

#get the info 
X = df_pivot.values

# standardize the features
X_scaled = StandardScaler().fit_transform(X)

# Initialize PCA
pca = PCA()

# Fit PCA to  the scaled data
pca.fit(X_scaled)

# Create a dataframe w/ the cumulative explained variance
explained_variance_df = pd.DataFrame({
    'Component': range(1, len(pca.explained_variance_ratio_) + 1),
    'Cumulative Explained Variance': np.cumsum(pca.explained_variance_ratio_)
})


cumulative_variance_chart = alt.Chart(explained_variance_df).mark_line(point=True).encode(
    alt.X('Component:O').title('Components'),
    y='Cumulative Explained Variance:Q'
).properties(
    title='Cumulative Explained Variance by Different Principal Components'
)

cumulative_variance_chart.display()


#Apply PCA!


# Pivot the DataFrame to get one row per patient with 1024 feature columns
features = df.pivot(index='PatientID', columns='FeatureIndex', values='FeatureValue')

#features data frame now has one row per patient and 1024 columns for features

# now standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Perform PCA
n_components = 100  # The number of components to keep, based on above chart
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(features_scaled)

# Create a DataFrame with the principal components
principal_df = pd.DataFrame(data=principal_components,
                            index=features.index,
                            columns=[f'PC{i}' for i in range(1, n_components + 1)])

# Reset index to bring PatientID back as a column
principal_df.reset_index(inplace=True)


# Now, extract the unique 'PatientID' and 'Diagnosis' from the original df
patient_info = df[['PatientID', 'Diagnosis']].drop_duplicates().set_index('PatientID')

# Merge the patient info and the principal components dataframes
final_df = patient_info.join(principal_df.set_index('PatientID')).reset_index()

print(final_df)
principal_df.head()


#Apply t-SNE!


# Select the feature columns for t-SNE
feature_columns = [f'PC{i}' for i in range(1, n_components + 1)]

# Extract feature data
feature_data = final_df[feature_columns]

# Apply t-SNE 
n_components_tsne = 2  #2 components for viz 
tsne = TSNE(n_components=n_components_tsne, random_state=42)
tsne_results = tsne.fit_transform(feature_data)

tsne_df = pd.DataFrame(data=tsne_results, columns=[f't-SNE{i}' for i in range(1, n_components_tsne + 1)])

tsne_final_df = pd.concat([final_df[['PatientID', 'Diagnosis']].reset_index(drop=True), tsne_df], axis=1)

scatter_plot = alt.Chart(tsne_final_df).mark_circle(size=60).encode(
    x='t-SNE1',
    y='t-SNE2',
    color='Diagnosis:N',  # Color by Diagnosis
    tooltip=['PatientID', 'Diagnosis']
).properties(
    width=500,
    height=400
)

scatter_plot



In [ ]:
#Produces chart of model accuracy with different parameters (as high as 85%)
import torch
import numpy as np
import pandas as pd
import altair as alt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from classifier import LinearClassifierAlexNet

#Choose only ones not used for training

ADNI_data = pd.read_csv('ADNI_Features_1-3v4.csv')
df = ADNI_data[ADNI_data['source']!='train']
df

# Don't want non-numeric columns
numeric_df = df.select_dtypes(include=[float, int]).dropna()

# Apply PCA
pca = PCA(n_components=0.999)  # Preserve 99% of variance
pca_result = pca.fit_transform(numeric_df)
pca_df = pd.DataFrame(data=pca_result)
pca_df = pd.concat([df['Diagnosis'], pca_df], axis=1)


# Function to run the model with different parameters
def run_model(X, y_encoded, loss_function, optimizer_function, lr, pca_components):
    # PCA transformation
    pca = PCA(n_components=pca_components)
    X_pca = pca.fit_transform(X)

    # Splitting the dataset
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    # Initialize the classifier
    in_dim = X_train.shape[1]
    model = LinearClassifierAlexNet(in_dim)

    # Loss and optimizer
    criterion = loss_function()
    optimizer = optimizer_function(model.parameters(), lr=lr)

    # Training loop
    epochsmax = 50


    accuracy = []
    for epoch in range(epochsmax):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Test the model
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            outputs = model(X_test)
            _, predicted = torch.max(outputs.data, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
        accuracy.append(correct / total * 100)

    return accuracy

# Load and preprocess the data
pca_df = pca_df.dropna()
X = pca_df.drop(columns=['Diagnosis'])
y = pca_df['Diagnosis']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Parameters to try
loss_functions = [torch.nn.CrossEntropyLoss, torch.nn.NLLLoss]
optimizer_functions = [torch.optim.Adam, torch.optim.SGD]
learning_rates = [0.01, 0.001, 0.0001]
pca_components_list = [0.9, 0.95, 0.99, 0.999]

# Running the model with different parameters
results = []
for loss_function in loss_functions:
    for optimizer_function in optimizer_functions:
        for lr in learning_rates:
            for pca_components in pca_components_list:
                accuracy = run_model(X, y_encoded, loss_function, optimizer_function, lr, pca_components)
                for epoch, acc in enumerate(accuracy):
                    results.append({
                        'Epoch': epoch,
                        'Accuracy': acc,
                        'Loss Function': loss_function.__name__,
                        'Optimizer': optimizer_function.__name__,
                        'Learning Rate': lr,
                        'PCA Components': pca_components
                    })

results_df = pd.DataFrame(results)

chart = alt.Chart(results_df).mark_circle().encode(
    x='Epoch:Q',
    y=alt.Y('Accuracy:Q', scale=alt.Scale(domain=[0, 100])),
    color='Learning Rate:N',
    tooltip=['Epoch', 'Accuracy', 'Loss Function', 'Optimizer', 'Learning Rate', 'PCA Components']
).properties(
    width=180,
    height=150
)

# Create small multiples
chart = chart.facet(
    column='Loss Function:N',
    row='Optimizer:N'
).resolve_scale(
    x='independent',
    y='independent'
).properties(
    title="Linear Classifier Accuracy"
)
chart = chart.configure_title(
    align='center'
)

chart

In [ ]:
# @title Grid Search Compare AUC
# @markdown Stochastic Gradient Descent w/ Cross Entropy Loss and batch size of 64. Compares for both LinearClassifierAlexNet and AdversarialClassifier from classifier.py number of hidden layers, learning rates, momentum, number of pca components, and number of epochs.
# WARNING: RUN AT YOUR OWN RISK, TAKES 3-4HOURS
import itertools
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#Increase reproducibility
seed = 20021102
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df = pd.read_csv('ADNI_Features_1-3v4.csv').dropna()
df = df[(df['Diagnosis'] == 1.0) | (df['Diagnosis'] == 3.0)] #leave out MCI but left in model training data for now

from classifier import AdversarialClassifier
from classifier import LinearClassifierAlexNet

Y = df['Diagnosis']
X = df.drop(columns=['Diagnosis', 'scan_ID', 'source', 'SubjectID', 'SessionId', 'DiagnosisVisDate', 'UniqueID'])

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)

#Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

#map diagnoses
y_train = y_train.map({1.0: 0, 3.0: 1})
y_val = y_val.map({1.0: 0, 3.0: 1})
y_test = y_test.map({1.0: 0, 3.0: 1})


# Convert to PyTorch tensors
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val = torch.tensor(y_val.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)


#DataLoader for validation
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(dataset=val_data, batch_size=64)

from classifier import AdversarialClassifier
from classifier import LinearClassifierAlexNet

train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


# Define parameter grids
param_grid_AdversarialClassifier = {
    'nhid': [100, 200, 300, 400],
    'lr': [0.0001, 0.001, 0.01],
    'momentum': [0.8, 0.9, 0.93, 0.95, 0.99],
    'num_epochs' : [100, 125, 150, 175, 200, 300, 400, 500],
    'pca_components': [0.9, 0.95, 0.99, None]
}

param_grid_LinearClassifierAlexNet = {
    'n_hid': [100, 200, 300, 400],
    'lr': [0.0001, 0.001, 0.01],
    'momentum': [0.8, 0.9, 0.93, 0.95, 0.99],
    'num_epochs' : [100, 125, 150, 175, 200, 300, 400, 500],
    'pca_components': [0.9, 0.95, 0.99, None]
}

def apply_pca(X_train, X_val, n_components):
    if n_components is None:
        return X_train, X_val  # No PCA applied

    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)  # Transform validation data using training PCA fit
    return X_train_pca, X_val_pca


def train_evaluate_model(model, train_loader, val_loader, num_epochs, lr, momentum):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(probabilities)

                # Check for NaNs in predictions
                if np.isnan(probabilities).any():
                    raise ValueError("NaN found in model predictions")

    if np.isnan(all_labels).any():
        raise ValueError("NaN found in validation labels")

    auc_score = roc_auc_score(all_labels, all_preds)
    return auc_score


def grid_search(param_grid, classifier_type, X_train, y_train, X_val, y_val):
    best_auc = 0
    best_params = {}

    for params in itertools.product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        # Apply PCA with the number of components set
        X_train_pca, X_val_pca = apply_pca(X_train, X_val, param_dict['pca_components'])

        # Adjust in_dim for the PCA-transformed data
        in_dim = X_train_pca.shape[1]

        # Make Dataloaders for the PCA data
        train_data_pca = TensorDataset(torch.tensor(X_train_pca, dtype=torch.float32), y_train)
        train_loader_pca = DataLoader(dataset=train_data_pca, batch_size=64, shuffle=True)
        val_data_pca = TensorDataset(torch.tensor(X_val_pca, dtype=torch.float32), y_val)
        val_loader_pca = DataLoader(dataset=val_data_pca, batch_size=64)

        # Initialize the model with the adjusted in_dim
        if classifier_type == 'AdversarialClassifier':
            model = AdversarialClassifier(in_dim=in_dim, nhid=param_dict['nhid'], out_dim=2)
        elif classifier_type == 'LinearClassifierAlexNet':
            model = LinearClassifierAlexNet(in_dim=in_dim, n_hid=param_dict['n_hid'], n_label=2)


        # Train and evaluate the model
        auc_score = train_evaluate_model(model, train_loader_pca, val_loader_pca, param_dict['num_epochs'], param_dict['lr'], param_dict['momentum'])
        print(auc_score)


        if auc_score > best_auc:
            best_auc = auc_score
            best_params = param_dict
            

    return best_params, best_auc

best_params_AdversarialClassifier, best_auc_AdversarialClassifier = grid_search(
    param_grid_AdversarialClassifier, 'AdversarialClassifier', X_train.numpy(), y_train, X_val.numpy(), y_val)

best_params_LinearClassifierAlexNet, best_auc_LinearClassifierAlexNet = grid_search(
    param_grid_LinearClassifierAlexNet, 'LinearClassifierAlexNet', X_train.numpy(), y_train, X_val.numpy(), y_val)

print("Best Params for AdversarialClassifier:", best_params_AdversarialClassifier, "with AUC:", best_auc_AdversarialClassifier)
print("Best Params for LinearClassifierAlexNet:", best_params_LinearClassifierAlexNet, "with AUC:", best_auc_LinearClassifierAlexNet)

In [ ]:
# @title AUC
# @markdown let's try some more combinations. and also let's increase test set size ... hopefully that was the issue and not overfitting :)

import itertools
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import random
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn

#Increase reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df = pd.read_csv('ADNI_Features_1-3v4.csv').dropna()
df = df[(df['Diagnosis'] == 1.0) | (df['Diagnosis'] == 3.0)] #leave out MCI but left in model training data for now
#df = df[df['source']!='train'] #take out model training data but leave val (not ideal)

from classifier import AdversarialClassifier
from classifier import LinearClassifierAlexNet

Y = df['Diagnosis']
X = df.drop(columns=['Diagnosis', 'scan_ID', 'source', 'SubjectID', 'SessionId', 'DiagnosisVisDate', 'UniqueID'])

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

#map diagnoses
y_train = y_train.map({1.0: 0, 3.0: 1})
y_val = y_val.map({1.0: 0, 3.0: 1})
y_test = y_test.map({1.0: 0, 3.0: 1})


# Convert to pytorch tensors
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val = torch.tensor(y_val.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)


#DataLoader for validation
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(dataset=val_data, batch_size=64)



train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


# Define parameter grids
param_grid_AdversarialClassifier = {
    #'nhid': [200, 300, 400, 500],
    'nhid': [400, 500],
    'lr': [0.001, 0.01],
    #'momentum': [0.9, 0.93, 0.95, 0.99],
    'momentum': [0.93, 0.99],
    'num_epochs' : [60, 500, 600], #500 epochs was best for both classifiers earlier, maybe higher would be even better?
    #'pca_components': [0.9, 0.95, 0.99, None]
    'pca_components': [0.95]
}

param_grid_LinearClassifierAlexNet = {
    #'n_hid': [200, 400, 500],
    'n_hid': [400, 500],
    'lr': [0.001, 0.0001],
    'momentum': [0.93, 0.99],
    'num_epochs' : [60, 500, 600],
    #'pca_components': [0.9, 0.95, 0.99, None]
    'pca_components': [0.95]
}

def apply_pca(X_train, X_val, n_components):
    if n_components is None:
        return X_train, X_val  # No PCA applied

    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)  # Transform validation data using training PCA fit
    return X_train_pca, X_val_pca


def train_evaluate_model(model, train_loader, val_loader, num_epochs, lr, momentum):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(probabilities)

                # Check for nans in predictions
                if np.isnan(probabilities).any():
                    raise ValueError("NaN found in model predictions")
    #and labels
    if np.isnan(all_labels).any():
        raise ValueError("NaN found in validation labels")

    auc_score = roc_auc_score(all_labels, all_preds)
    return auc_score


def grid_search(param_grid, classifier_type, X_train, y_train, X_val, y_val):
    best_auc = 0
    k = 0
    best_params = {}

    for params in itertools.product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        # Apply PCA w the number of components set
        X_train_pca, X_val_pca = apply_pca(X_train, X_val, param_dict['pca_components'])

        # need to adjust in_dim for the PCA-transformed data
        in_dim = X_train_pca.shape[1]

        # Make dataloaders for the PCA data
        train_data_pca = TensorDataset(torch.tensor(X_train_pca, dtype=torch.float32), y_train)
        train_loader_pca = DataLoader(dataset=train_data_pca, batch_size=4, shuffle=True) #batch size of 4 this time let's see if that's better
        val_data_pca = TensorDataset(torch.tensor(X_val_pca, dtype=torch.float32), y_val)
        val_loader_pca = DataLoader(dataset=val_data_pca, batch_size=4)

        # Initialize  model with the adjusted in_dim
        if classifier_type == 'AdversarialClassifier':
            model = AdversarialClassifier(in_dim=in_dim, nhid=param_dict['nhid'], out_dim=2)
        elif classifier_type == 'LinearClassifierAlexNet':
            model = LinearClassifierAlexNet(in_dim=in_dim, n_hid=param_dict['n_hid'], n_label=2)


        # Train and eval the model
        auc_score = train_evaluate_model(model, train_loader_pca, val_loader_pca, param_dict['num_epochs'], param_dict['lr'], param_dict['momentum'])
        k += 1
        print(auc_score, best_auc, k, param_dict)


        if auc_score > best_auc:
            best_auc = auc_score
            best_params = param_dict
            

    return best_params, best_auc


best_params_LinearClassifierAlexNet, best_auc_LinearClassifierAlexNet = grid_search(
    param_grid_LinearClassifierAlexNet, 'LinearClassifierAlexNet', X_train.numpy(), y_train, X_val.numpy(), y_val)

best_params_AdversarialClassifier, best_auc_AdversarialClassifier = grid_search(
    param_grid_AdversarialClassifier, 'AdversarialClassifier', X_train.numpy(), y_train, X_val.numpy(), y_val)


print("Best Params for AdversarialClassifier:", best_params_AdversarialClassifier, "with AUC:", best_auc_AdversarialClassifier)
print("Best Params for LinearClassifierAlexNet:", best_params_LinearClassifierAlexNet, "with AUC:", best_auc_LinearClassifierAlexNet)

In [ ]:
AUC CALCULATION

#best_params_AdversarialClassifier =  {'nhid': 300, 'lr': 0.001, 'momentum': 0.93, 'num_epochs': 500, 'pca_components': 0.99} 
#best_params_LinearClassifierAlexNet=  {'n_hid': 100, 'lr': 0.01, 'momentum': 0.99, 'num_epochs': 500, 'pca_components': 0.9} 
 

#USING PARAMETERS FROM THE PAPER OF THE CNN (WHICH ACHIEVED 85.12 AUC):
best_params_AdversarialClassifier =  {'nhid': 200, 'lr': 0.01, 'momentum': 0.9, 'num_epochs': 60, 'pca_components': 0.95} 
best_params_LinearClassifierAlexNet=  {'n_hid': 200, 'lr': 0.01, 'momentum': 0.9, 'num_epochs': 60, 'pca_components': 0.95} 
 

#Evaluate test set with best parameters based on grid search
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Function to evaluate the model on the test set
def evaluate_test_set(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(probabilities)

    auc_score = roc_auc_score(all_labels, all_preds)
    return auc_score

# Apply PCA to the test set
X_train_pca, X_test_pca = apply_pca(X_train.numpy(), X_test.numpy(), best_params_AdversarialClassifier['pca_components'])

# Convert test set to PyTorch tensors
test_data = TensorDataset(torch.tensor(X_test_pca, dtype=torch.float32), y_test)
test_loader = DataLoader(dataset=test_data, batch_size=64)

# Initialize models with best parameters
best_model_AdversarialClassifier = AdversarialClassifier(in_dim=X_test_pca.shape[1], nhid=best_params_AdversarialClassifier['nhid'], out_dim=2)
best_model_LinearClassifierAlexNet = LinearClassifierAlexNet(in_dim=X_test_pca.shape[1], n_hid=best_params_LinearClassifierAlexNet['n_hid'], n_label=2)

# Evaluate AdversarialClassifier on test set
test_auc_AdversarialClassifier = evaluate_test_set(best_model_AdversarialClassifier, test_loader)
print("Test AUC for AdversarialClassifier:", test_auc_AdversarialClassifier)

# Evaluate LinearClassifierAlexNet on test set
test_auc_LinearClassifierAlexNet = evaluate_test_set(best_model_LinearClassifierAlexNet, test_loader)
print("Test AUC for LinearClassifierAlexNet:", test_auc_LinearClassifierAlexNet)

